In [ ]:
import os
from datasets import load_dataset
from pprint import pprint

if not os.environ.get("HF_TOKEN"):
    # FYI export HF_TOKEN=$(pbpaste)
    raise ValueError("You need to set HF_TOKEN environment variable. Get one from huggingface.co")

# Load the shell/bash subset
#   FYI:   https://huggingface.co/datasets/bigcode/the-stack/tree/main/data/shell (FYI gated, must give email and approve)
full_dataset = load_dataset( "bigcode/the-stack", split="train", data_dir="data/shell")  # , lang=["bash"])
# data_dir data/shell has 11 files, not bad size (about 4GB IIEC)

# Save locally in diff format if needed, I am going to stick with parquet
# full_dataset.to_csv("shell_scripts.csv")
# print(full_dataset.column_names)
subset = full_dataset.select(range(1))
print("shape", subset.shape)

original_columns = subset.column_names

In [2]:
# build corpus
if not os.path.exists("tmp"):
    os.makedirs("tmp")

corpus_file = "tmp/shell_scripts_corpus.sh"
if not os.path.exists(corpus_file):
    with open(corpus_file, "w") as f:
        for example in subset["content"]:  # Adjust "content" to match your dataset key
            f.write(example + "\n")

In [ ]:
from tokenizers import ByteLevelBPETokenizer

tokenizer_path = "tmp/trained-tokenizer"
if not os.path.exists(tokenizer_path):
    os.makedirs(tokenizer_path)

# Train the tokenizer
if not os.path.exists(tokenizer_path + "/vocab.json"):
    tokenizer = ByteLevelBPETokenizer()
    tokenizer.train(files=[corpus_file], vocab_size=8000, min_frequency=2)  # PRN adjust vocab_size/min_frequency?
    tokenizer.save_model(tokenizer_path)

# load the tokenizer
tokenizer = ByteLevelBPETokenizer(tokenizer_path + "/vocab.json", tokenizer_path + "/merges.txt")

# tokenize (FYI .map is immutable, a new dataset is returned with the added column)
print("before tokenize dataset size: ", subset.shape)
subset = subset.map(lambda x: {"tokens": tokenizer.encode(x["content"]).ids})
print("after tokenize dataset size: ", subset.shape)
print()
print(subset["tokens"][0])  # Example tokenized output
print()
#
# # VIEW SOME TOKENS:
# # show each token for first 10:
# for i in range(100):
#     print(subset_tokenizd["tokens"][0][i], tokenizer.decode([subset_tokenizd["tokens"][0][i]]))


In [ ]:
def create_training_pairs(tokens, seq_len):
    # Split into sequences of length seq_len + 1
    sequences = [tokens[i:i + seq_len + 1] for i in range(len(tokens) - seq_len)]
    return sequences

seq_len = 50
subset = subset.map(lambda x: {"sequences": create_training_pairs(x["tokens"], seq_len)})


In [ ]:
# INSPECTING SUBSET with new columns:
def print_subset():
    print(subset.shape)
    for name in subset.column_names:
        print(f"{name}: {subset[name]}")
        print(f"  {len(subset[name])}")
    print()


def print_subset_new_columns():
    for name in subset.column_names:
        if name in original_columns:
            continue
        print(f"{name}: {subset[name]}")
        print(f"  {len(subset[name])}")

    print()


print_subset()

print()
print("# SEQUENCES:")
print()
for i in range(3):
    # each record (in original dataset) now has a list of sequences (so, sequences is not a scalar value like content)
    print("## sequence " + str(i))
    print(subset["sequences"][0][i])
    print(tokenizer.decode(subset["sequences"][0][i]))
    print()

In [ ]:
# Separate input and target
def split_input_target(sequence):
    return sequence[:-1], sequence[1:]


def mapper(x):
    split = split_input_target(x["sequences"])
    return {"x_input": split[0], "y_target": split[1]}

subset = subset.map(mapper)


In [ ]:
print_subset_new_columns()

In [8]:
import torch
import torch.nn as nn


class CodeCompletionTransformer(nn.Module):

    def __init__(self, vocab_size, d_model, nhead, num_layers, seq_len):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = nn.Parameter(torch.zeros(1, seq_len, d_model))
        self.transformer = nn.Transformer(d_model, nhead, num_layers, batch_first=True)
        self.fc = nn.Linear(d_model, vocab_size)

    def forward(self, x):
        x = self.embedding(x) + self.positional_encoding
        x = self.transformer(x, x)  # Encoder-only transformer
        return self.fc(x)


# Initialize
vocab_size = 8000
model = CodeCompletionTransformer(vocab_size=vocab_size, d_model=128, nhead=4, num_layers=2, seq_len=seq_len)
